In [1]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/code/CT_MPS_mini/CT`


In [ ]:
Pkg.status()

Project CT v0.1.0
Status `~/code/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [6e4b80f9] BenchmarkTools v1.6.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.19
  [682c06a0] JSON v0.21.4
  [33e6dc65] MKL v0.9.0
  [da04e1cc] MPI v0.20.23
  [3da0fdf6] MPIPreferences v0.1.11
⌅ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.16
⌅ [a759f4b9] TimerOutputs v0.5.25
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
include("global_adder_passthrough.jl")

filter_highest_tag_number (generic function with 1 method)

In [4]:
using Pkg
Pkg.activate("CT")
using CT

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 32
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
_maxdim = 2^(div(L,2))
_maxdim0 = 2^10
_eps = 1e-10
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
shift_1_6_bits, shift_1_6_amount = fraction_to_binary_shift(1, 6, L)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _eps, _maxdim0);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# create_addition_tensor_with_carry(shift_bit::Int, s::Index, s_prime::Index, c_in::Index, c_out::Index)
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_6_bits, ram_phy);
i1 = 1;

  Activating project at `~/code/CT_MPS_mini/CT`


initializing random MPS with linkdims 1024


In [5]:
using .CT: all_bond_dim, max_bond_dim, maxlinkdim

In [ ]:
# function sort_indices(tensor::ITensor, _tags::String)
#     indices = filterinds(inds(tensor), _tags)
#     sorted_inds = sort(collect(indices), by = x -> begin
#         tag_str = string(tags(x))
#         m = match(r"c=(\d+)", tag_str)
#         parse(Int, m.captures[1])
#     end)
#     return sorted_inds
# end

# function lower_prime_level(tensor::ITensor, _tags::String)
#     # this function lower the double prime level indices to single prime level
#     for index in filterinds(inds(tensor), _tags)
#         if plev(index) > 1
#             tensor = prime(tensor, -(plev(index)-1), index)
#         end
#     end
#     return tensor
# end

# # set the boundary condition so that we are not carrying dangling indices around
# function build_adder_mpo(i1::Int,xj::Set,qubit_site::Vector{Index{Int64}},L::Int,phy_ram::Vector{Int},phy_list::Vector{Int})
#     lsb_tensor = onehot(carry_links[2]=>1)

#     # discard the carry out of the msb tensor
#     msb_tensor = ITensor(carry_links[1])
#     msb_tensor[carry_links[1]=>1] = 1.0
#     msb_tensor[carry_links[1]=>2] = 1.0

#     gate_vec[1] = gate_vec[1] * msb_tensor
#     gate_vec[2] = gate_vec[2] * lsb_tensor
#     mpo_vec = [ITensor() for _ in 1:L];

#     tmp_tensor = gate_vec[1] * prime(gate_vec[2], "Qubit,Site,n=2") * prime(gate_vec[3], "Qubit,Site,n=3")
#     tmp_tensor = lower_prime_level(tmp_tensor, "Qubit,Site")

#     U, S, V = svd(tmp_tensor, [qubit_site[1], prime(qubit_site[1])], lefttags = "Link,l=1", cutoff=_eps, maxdim=_maxdim);

#     mpo_vec[1] = U
#     i1 = 4
#     while i1 <= length(gate_vec)
#         println("i1 = ", i1)
#         tmp_tensor = lower_prime_level(S * V * prime(gate_vec[i1], "Qubit,Site,n=$(i1)"), "Qubit,Site")
#         U, S, V = svd(tmp_tensor, [qubit_site[i1-2], prime(qubit_site[i1-2]), filterinds(tmp_tensor, tags="Link,l=$(i1-3)")...], lefttags = "Link,l=$(i1-2)", cutoff=_eps, maxdim=_maxdim);
#         # println(filter(x -> x >= 1e-10, vec(array(S))))
#         mpo_vec[i1-2] = U
#         i1 += 1
#     end

#     U, S, V = svd(S * V, [qubit_site[end-2], prime(qubit_site[end-2]), filterinds(S * V, tags="Link,l=$(L-3)")...], lefttags = "Link,l=$(L-2)", cutoff=_eps, maxdim=_maxdim);
#     mpo_vec[L-2] = U
#     U, S, V = svd(lower_prime_level(S * V * prime(T_vec[L], "Qubit,Site,n=$(L)"), "Qubit,Site"), [qubit_site[L-1], prime(qubit_site[L-1])], lefttags = "Link,l=$(L-1)", cutoff=_eps, maxdim=_maxdim);
#     mpo_vec[L-1] = U
#     mpo_vec[L] = S * V

#     return MPO(mpo_vec)

# end

lower_prime_level (generic function with 1 method)

In [8]:
mpo_test = build_adder_mpo(i1,xj,qubit_site,L,phy_ram,phy_list)

i1 = 4
i1 = 5
i1 = 6
i1 = 7
i1 = 8
i1 = 9
i1 = 10
i1 = 11
i1 = 12
i1 = 13
i1 = 14
i1 = 15
i1 = 16
i1 = 17
i1 = 18
i1 = 19
i1 = 20
i1 = 21
i1 = 22
i1 = 23
i1 = 24
i1 = 25
i1 = 26
i1 = 27
i1 = 28
i1 = 29
i1 = 30
i1 = 31


MPO
[1] ((dim=2|id=522|"Qubit,Site,n=1"), (dim=2|id=522|"Qubit,Site,n=1")', (dim=2|id=405|"Link,l=1"))
[2] ((dim=2|id=405|"Link,l=1"), (dim=2|id=770|"Qubit,Site,n=2"), (dim=2|id=770|"Qubit,Site,n=2")', (dim=4|id=827|"Link,l=2"))
[3] ((dim=4|id=827|"Link,l=2"), (dim=2|id=433|"Qubit,Site,n=3"), (dim=2|id=433|"Qubit,Site,n=3")', (dim=4|id=418|"Link,l=3"))
[4] ((dim=4|id=418|"Link,l=3"), (dim=2|id=950|"Qubit,Site,n=4"), (dim=2|id=950|"Qubit,Site,n=4")', (dim=4|id=943|"Link,l=4"))
[5] ((dim=4|id=943|"Link,l=4"), (dim=2|id=22|"Qubit,Site,n=5"), (dim=2|id=22|"Qubit,Site,n=5")', (dim=4|id=21|"Link,l=5"))
[6] ((dim=4|id=21|"Link,l=5"), (dim=2|id=35|"Qubit,Site,n=6"), (dim=2|id=35|"Qubit,Site,n=6")', (dim=4|id=24|"Link,l=6"))
[7] ((dim=4|id=24|"Link,l=6"), (dim=2|id=632|"Qubit,Site,n=7"), (dim=2|id=632|"Qubit,Site,n=7")', (dim=4|id=537|"Link,l=7"))
[8] ((dim=4|id=537|"Link,l=7"), (dim=2|id=250|"Qubit,Site,n=8"), (dim=2|id=250|"Qubit,Site,n=8")', (dim=4|id=971|"Link,l=8"))
[9] ((dim=4|id=971|"Lin

In [ ]:
apply(mpo_test,initial_state)
@time apply(mpo_test,initial_state)
apply(add1_3,initial_state)
@time apply(add1_3,initial_state)
# maxbond_L = []
# maxbond_3_L = []
# maxbond_6_L = []
# for L in [8,12,16,20,24,28,32,36,40]
#     # initialize haining adder mpo
#     ct=CT.CT_MPS(L=L,seed=0,seed_C=0,seed_m=0,folded=true,store_op=true,store_vec=false,ancilla=0,_maxdim0=60,xj=Set([1 // 3, 2 // 3]),debug=true,simplified_U=true, x0=8//2^12,)
#     maxbond = []

#     for adder in ct.adder
#         push!(maxbond, maxlinkdim(adder))
#     end
#     push!(maxbond_L, maximum(maxbond))
#     println("L = $L")

#     qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)

#     maxbond_3 = []
#     maxbond_6 = []
#     for i in 1:L
#         push!(maxbond_3, maxlinkdim(add1_3))
#         push!(maxbond_6, maxlinkdim(add1_6))
#     end
#     # initialize haining adder mpo
#     add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
#     add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])
#     push!(maxbond_3_L, maximum(maxbond_3))
#     push!(maxbond_6_L, maximum(maxbond_6))
# end


In [ ]:
# maxbond_L, maxbond_3_L, maxbond_6_L # you can see that the fixing spurs and projections ramp up the maximum bond dimension considerably.

In [ ]:
# maxbond_L_test = []
# for L in [8,12,16,20,24,28,32,36,40]
#     # initialize haining adder mpo
#     maxbond = []

#     qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)

#     for i1 in 1:L
#         push!(maxbond, maxlinkdim(adder_MPO_test(i1,Set([1 // 3, 2 // 3]),qubit_site,L,phy_ram,phy_list)))
#     end
#     push!(maxbond_L_test, maximum(maxbond))
#     println("L = $L")

# end
